In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 행렬 분해를 이용한 잠재 요인 협업 필터링 실습

행렬은 괄호 안에 수 또는 문자를 행과 열에 맞게 나열한 것으로, 이때 수 또는 문자를 ‘행렬의 성분이라 합니다. 행렬은 인공지능 연구를 위한 언어이자, 빅데이터를 손쉽게 다루기 위한 필수 도구입니다. 인공지능의 기반인 컴퓨터가 어떤 문제를 풀도록 하려면 문제를 행렬이 포함된 식으로 바꾸는 과정이 반드시 필요하기 때문인데요, 예를 들어 복잡한 3차 미분방정식_보통 방정식은 종류에 상관없이 차수가 올라갈수록 풀기 어렵다을 풀 때, 이를 간단한 1차 미분방정식 3개로 표현해서(선형화) 행렬로 나타내면 컴퓨터를 통해 쉽게 답을 구할 수 있습니다.



그런데 행렬의 크기_행의 수와 열의 수를 곱한 값가 100만×100만 처럼 큰 경우에는 슈퍼 컴퓨터조차 문제를 푸는데 굉장히 오랜 시간이 걸립니다. 이것을 해결하려면 크기가 큰 행렬을 컴퓨터가 효과적으로 다룰 수 있도록 해주는 '선형대수학'의 이론을 이용해야 합니다.

행렬의 성질을 연구하는 선형대수학은 크기가 큰 행렬을 컴퓨터가 효과적으로 다루도록 해주는 마법의 도구입니다. 푸는 데 10년 이상 걸릴 문제도 선형대수학를 이용하면 순식간에 답을 내놓기도 하죠.



'빅데이터'를 분석하는 것을 수학적으로 살펴보면 무수히 많은 데이터 중 유의미한 데이터를 추출해 최적의 행렬을 구성한 뒤, 행렬을 적절하게 다뤄 필요한 결과 값을 도출해 내는 것입니다. 알파고에서 쓰인 인공지능 기술, 딥러닝은 인간의 뇌를 모사한 인공 신경망에서 발전된 예측･분류 기법으로 이미지 인식, 음성 인식 등에 활용되며 최근에는 사람을 뛰어넘는 성능을 보였습니다.



이러한 대부분의 딥러닝 계산은 행렬 곱 연산을 포함한 기본적인 행렬 연산을 통해 수행됩니다. '인공지능, 수학으로 타파' 첫 시간에는 코딩 명령어를 이용해 행렬을 만들고, 이를 통해 다양한 문제들을 해결해 보도록 해요!

http://www.polymath.co.kr/contents/view/18142

#### 예제
- 원본행렬 R을 P와 Q로 분해한 뒤에 다시 P와 Q.T의 내적으로 예측 행렬을 만드는 예제
- 잠재 요인 차원 K는 3으로 설정
#### 순서
- R에서 널 값을 제외한 데이터의 행렬 인덱스를 추출함

In [ ]:
# 4 X 5 행렬
# R = P*Q.T
# P - 4 X 3
# Q - 5 X 3 (이것의 Q.T가 3 X 5 임)

import numpy as np

# 원본 행렬 R 생성, 분해 행렬 P와 Q 초기화, 잠재요인 차원 K는 3 설정.
R = np.array([[4, np.NaN, np.NaN, 2, np.NaN ],
              [np.NaN, 5, np.NaN, 3, 1 ],
              [np.NaN, np.NaN, 3, 4, 4 ],
              [5, 2, 1, 2, np.NaN ]])
num_users, num_items = R.shape
K=3

# P와 Q 매트릭스의 크기를 지정하고 정규분포를 가진 random한 값으로 입력합니다.
np.random.seed(1)
P = np.random.normal(scale=1./K, size=(num_users, K))
Q = np.random.normal(scale=1./K, size=(num_items, K))

In [ ]:
# 실제 R 행렬과 예측 행렬의 오차를 구하는 get_rmse() 함수 만들기
#  mean_squared_error(정답, 시험문제 푼 것)
# x_non_zero_ind에는 non_zeros의 행 인덱스가 담김 [0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3]
# y_non_zero_ind에는 non_zeros의 열 인덱스가 담김 [0, 3, 1, 3, 4, 2, 3, 4, 0, 1, 2, 3]
# R_non_zeros에는 R행렬의 0이 아닌 값들만 리스트 형태로 저장됨 [4, 2, 5, 3, 1, 3, 4, 4, 5, 2, 1, 2]
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0    # 함수 시작시 변수 초기화 , 추후 오차 계산을 위해 사용됨
    # 두개의 분해된 행렬 P와 Q.T의 내적으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P, Q.T)

    # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스 추출하여 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)

    return rmse

In [ ]:
R

array([[ 4., nan, nan,  2., nan],
       [nan,  5., nan,  3.,  1.],
       [nan, nan,  3.,  4.,  4.],
       [ 5.,  2.,  1.,  2., nan]])

In [ ]:
full_pred_matrix_non_zeros
[3.9912639  1.99893754 4.97751915 2.98549238 1.0039134  2.98766484
 3.97918846 3.98527077 4.97197974 2.00367132 1.00479323 2.00565351]

In [ ]:
non_zeros

[(0, 0, 4.0),
 (0, 3, 2.0),
 (1, 1, 5.0),
 (1, 3, 3.0),
 (1, 4, 1.0),
 (2, 2, 3.0),
 (2, 3, 4.0),
 (2, 4, 4.0),
 (3, 0, 5.0),
 (3, 1, 2.0),
 (3, 2, 1.0),
 (3, 3, 2.0)]

In [ ]:
# R > 0 인 행 위치, 열 위치, 값을 non_zeros 리스트에 저장.
non_zeros = [ (i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0 ]

steps=1000    # 1,000번 동안 반복하면서 P, Q 값 업데이트 함
learning_rate=0.01
r_lambda=0.01

# SGD 기법으로 P와 Q 매트릭스를 계속 업데이트.
for step in range(steps):
    for i, j, r in non_zeros:
        # 실제 값과 예측 값의 차이인 오류 값 구함
        eij = r - np.dot(P[i, :], Q[j, :].T)
        # Regularization을 반영한 SGD 업데이트 공식 적용
        P[i,:] = P[i,:] + learning_rate*(eij * Q[j, :] - r_lambda*P[i,:])
        Q[j,:] = Q[j,:] + learning_rate*(eij * P[i, :] - r_lambda*Q[j,:])

    rmse = get_rmse(R, P, Q, non_zeros)
    if (step % 50) == 0 :    # 50회 마다 오류값 출력함
        print("### iteration step : ", step," rmse : ", rmse)

### iteration step :  0  rmse :  0.013954052427206728
### iteration step :  50  rmse :  0.013947111666906707
### iteration step :  100  rmse :  0.013940264575968541
### iteration step :  150  rmse :  0.013933509627427575
### iteration step :  200  rmse :  0.013926845324152191
### iteration step :  250  rmse :  0.013920270198168406
### iteration step :  300  rmse :  0.01391378280998507
### iteration step :  350  rmse :  0.013907381747954138
### iteration step :  400  rmse :  0.013901065627631203
### iteration step :  450  rmse :  0.013894833091163608
### iteration step :  500  rmse :  0.013888682806688372
### iteration step :  550  rmse :  0.013882613467747702
### iteration step :  600  rmse :  0.013876623792719842
### iteration step :  650  rmse :  0.013870712524266174
### iteration step :  700  rmse :  0.013864878428783034
### iteration step :  750  rmse :  0.013859120295882179
### iteration step :  800  rmse :  0.013853436937868202
### iteration step :  850  rmse :  0.013847827189245

In [ ]:
pred_matrix = np.dot(P, Q.T)
print('예측 행렬:\n', np.round(pred_matrix, 3))

예측 행렬:
 [[3.991 0.897 1.306 2.002 1.663]
 [6.696 4.978 0.979 2.981 1.003]
 [6.677 0.391 2.987 3.977 3.986]
 [4.968 2.005 1.006 2.017 1.14 ]]


### 행렬 분해 기반의 잠재 요인 협업 필터링 실습

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    # 두개의 분해된 행렬 P와 Q.T의 내적 곱으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P, Q.T)

    # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스 추출하여 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]

    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]

    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)

    return rmse

In [ ]:
def matrix_factorization(R, K, steps=200, learning_rate=0.01, r_lambda = 0.01):
    num_users, num_items = R.shape
    # P와 Q 매트릭스의 크기를 지정하고 정규분포를 가진 랜덤한 값으로 입력합니다.
    np.random.seed(1)
    P = np.random.normal(scale=1./K, size=(num_users, K))
    Q = np.random.normal(scale=1./K, size=(num_items, K))

    # R > 0 인 행 위치, 열 위치, 값을 non_zeros 리스트 객체에 저장.
    non_zeros = [ (i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0 ]

    # SGD기법으로 P와 Q 매트릭스를 계속 업데이트.
    for step in range(steps):
        for i, j, r in non_zeros:
            # 실제 값과 예측 값의 차이인 오류 값 구함
            eij = r - np.dot(P[i, :], Q[j, :].T)
            # Regularization을 반영한 SGD 업데이트 공식 적용
            P[i,:] = P[i,:] + learning_rate*(eij * Q[j, :] - r_lambda*P[i,:])
            Q[j,:] = Q[j,:] + learning_rate*(eij * P[i, :] - r_lambda*Q[j,:])

        rmse = get_rmse(R, P, Q, non_zeros)
        if (step % 10) == 0 :
            print("### iteration step : ", step," rmse : ", rmse)

    return P, Q

In [ ]:
import pandas as pd
import numpy as np

movies = pd.read_csv('../dataset/ml-latest-small/movies.csv')
ratings = pd.read_csv('../dataset/ml-latest-small/ratings.csv')
ratings = ratings[['userId', 'movieId', 'rating']]
ratings_matrix = ratings.pivot_table('rating', index='userId', columns='movieId')

# title 컬럼을 얻기 이해 movies 와 조인 수행
rating_movies = pd.merge(ratings, movies, on='movieId')

# columns='title' 로 title 컬럼으로 pivot 수행.
ratings_matrix = rating_movies.pivot_table('rating', index='userId', columns='title')

In [ ]:
P, Q = matrix_factorization(ratings_matrix.values, K=50, steps=200, learning_rate=0.01, r_lambda = 0.01)
pred_matrix = np.dot(P, Q.T)

### iteration step :  0  rmse :  2.9023619751336867
### iteration step :  10  rmse :  0.7335768591017927
### iteration step :  20  rmse :  0.5115539026853442
### iteration step :  30  rmse :  0.37261628282537446
### iteration step :  40  rmse :  0.2960818299181014
### iteration step :  50  rmse :  0.2520353192341642
### iteration step :  60  rmse :  0.22487503275269854
### iteration step :  70  rmse :  0.20685455302331537
### iteration step :  80  rmse :  0.19413418783028685
### iteration step :  90  rmse :  0.18470082002720403
### iteration step :  100  rmse :  0.17742927527209104
### iteration step :  110  rmse :  0.1716522696470749
### iteration step :  120  rmse :  0.1669518194687172
### iteration step :  130  rmse :  0.16305292191997542
### iteration step :  140  rmse :  0.15976691929679643
### iteration step :  150  rmse :  0.1569598699945732
### iteration step :  160  rmse :  0.15453398186715428
### iteration step :  170  rmse :  0.15241618551077643
### iteration step :  180  rm

In [ ]:
ratings_pred_matrix = pd.DataFrame(data=pred_matrix, index= ratings_matrix.index,
                                   columns = ratings_matrix.columns)

ratings_pred_matrix.head(3)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,3.055084,4.092018,3.564130,4.502167,3.981215,1.271694,3.603274,2.333266,5.091749,3.972454,...,1.402608,4.208382,3.705957,2.720514,2.787331,3.475076,3.253458,2.161087,4.010495,0.859474
2,3.170119,3.657992,3.308707,4.166521,4.311890,1.275469,4.237972,1.900366,3.392859,3.647421,...,0.973811,3.528264,3.361532,2.672535,2.404456,4.232789,2.911602,1.634576,4.135735,0.725684
3,2.307073,1.658853,1.443538,2.208859,2.229486,0.780760,1.997043,0.924908,2.970700,2.551446,...,0.520354,1.709494,2.281596,1.782833,1.635173,1.323276,2.887580,1.042618,2.293890,0.396941


In [ ]:
def get_unseen_movies(ratings_matrix, userId):
    # userId로 입력받은 사용자의 모든 영화정보 추출하여 Series로 반환함.
    # 반환된 user_rating 은 영화명(title)을 index로 가지는 Series 객체임.
    user_rating = ratings_matrix.loc[userId,:]

    # user_rating이 0보다 크면 기존에 관람한 영화임. 대상 index를 추출하여 list 객체로 만듬
    already_seen = user_rating[ user_rating > 0].index.tolist()

    # 모든 영화명을 list 객체로 만듬.
    movies_list = ratings_matrix.columns.tolist()

    # list comprehension으로 already_seen에 해당하는 movie는 movies_list에서 제외함.
    unseen_list = [ movie for movie in movies_list if movie not in already_seen]

    return unseen_list

In [ ]:
def recomm_movie_by_userid(pred_df, userId, unseen_list, top_n=10):
    # 예측 평점 DataFrame에서 사용자id index와 unseen_list로 들어온 영화명 컬럼을 추출하여
    # 가장 예측 평점이 높은 순으로 정렬함.
    recomm_movies = pred_df.loc[userId, unseen_list].sort_values(ascending=False)[:top_n]
    return recomm_movies

In [ ]:
# 사용자가 관람하지 않는 영화명 추출
unseen_list = get_unseen_movies(ratings_matrix, 9)

# 아이템 기반의 인접 이웃 협업 필터링으로 영화 추천
recomm_movies = recomm_movie_by_userid(ratings_pred_matrix, 9, unseen_list, top_n=10)

# 평점 데이타를 DataFrame으로 생성.
recomm_movies = pd.DataFrame(data=recomm_movies.values,index=recomm_movies.index,columns=['pred_score'])
recomm_movies